# SQL Event-based data:

## Objetivos:
    1. Utilizar un pandas data-frame como base de datos.
    2. Enviar un query utilizando pandas.
    3. Event-based data: propiedades, características y retos.
    4. Consideraciones: Subqueries & partitions.

## 1. Librerias y configuraciones

In [20]:
import pandas as pd
import duckdb

In [2]:
duck_query = lambda q: duckdb.query(q)

## 2. Entendiendo la data de eventos

In [16]:
event_df = pd.read_csv("events.csv")

In [21]:
event_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [22]:
event_df[event_df["transactionid"].notna()].head()

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0
814,1433176736375,102019,transaction,150318,13556.0
843,1433174518180,189384,transaction,310791,7244.0


## 3. Trabajando con SQLDF

In [18]:
duck_query(
    """
    SELECT *
    FROM event_df
    LIMIT 1
    ;
    """
)

┌───────────────┬───────────┬─────────┬────────┬───────────────┐
│   timestamp   │ visitorid │  event  │ itemid │ transactionid │
│     int64     │   int64   │ varchar │ int64  │    double     │
├───────────────┼───────────┼─────────┼────────┼───────────────┤
│ 1433221332117 │    257597 │ view    │ 355908 │          NULL │
└───────────────┴───────────┴─────────┴────────┴───────────────┘

In [19]:
duck_query(
    """
    SELECT
        a.event
        ,COUNT(DISTINCT visitorid) visits
        ,COUNT(DISTINCT itemid) items
        ,COUNT(DISTINCT transactionid) txns
    FROM event_df a
    GROUP BY 1
    ORDER BY 2 DESC
    ;
    """
)

┌─────────────┬─────────┬────────┬───────┐
│    event    │ visits  │ items  │ txns  │
│   varchar   │  int64  │ int64  │ int64 │
├─────────────┼─────────┼────────┼───────┤
│ view        │ 1404179 │ 234838 │     0 │
│ addtocart   │   37722 │  23903 │     0 │
│ transaction │   11719 │  12025 │ 17672 │
└─────────────┴─────────┴────────┴───────┘

In [13]:
mysql("""
    SELECT
        visitorid
        ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
        ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
        ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
    FROM event_df
    GROUP BY 1
    LIMIT 10
    ;
""")

,visitorid,views,cart,txn
0,0,1,0,0
1,1,1,0,0
2,2,1,0,0
3,3,1,0,0
4,4,1,0,0
5,5,1,0,0
6,6,1,1,0
7,7,1,0,0
8,8,1,0,0
9,9,1,0,0


In [7]:
mysql("""
    SELECT
        COUNT(DISTINCT CASE WHEN views = 1 THEN visitorid END) visits
        ,COUNT(DISTINCT CASE WHEN views = 1 AND cart = 1 THEN visitorid END) cart
        ,COUNT(DISTINCT CASE WHEN views = 1 AND cart = 1 AND txn = 1 THEN visitorid END) txns
    FROM (
        SELECT
            visitorid
            ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
            ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
            ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
        FROM event_df
        GROUP BY 1
    ) a
    ;
""")

,visits,cart,txns
0,1404179,34401,10228


In [16]:
mysql("""
    SELECT 
        COUNT(DISTINCT visitorid) vistors_with_txns
    FROM event_df
    WHERE 
        event = 'transaction'
    ;
""")

,vistors_with_txns
0,11719


In [17]:
mysql("""
    SELECT 
        views
        ,cart
        ,COUNT(DISTINCT visitorid) visits
    FROM (
        SELECT
            visitorid
            ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
            ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
            ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
        FROM event_df
        GROUP BY 1
    ) a
    WHERE
        txn = 1
    GROUP BY 1,2
    ;
""")

,views,cart,visits
0,0,0,80
1,0,1,348
2,1,0,1063
3,1,1,10228


In [31]:
mysql("""
    SELECT
        visitorid
        ,event
        ,timestamp
        ,itemid
        ,transactionid
        ,ROW_NUMBER() OVER(PARTITION BY visitorid, event ORDER BY timestamp) seq
    FROM event_df
    GROUP BY 1,2,3,4,5
    LIMIT 10
    ;
""")

,visitorid,event,timestamp,itemid,transactionid,seq
0,0,view,1442004589439,285930,None,1
1,0,view,1442004759591,357564,None,2
2,0,view,1442004917175,67045,None,3
3,1,view,1439487966444,72028,None,1
4,2,view,1438969904567,325215,None,1
5,2,view,1438970013790,325215,None,2
6,2,view,1438970212664,259884,None,3
7,2,view,1438970468920,216305,None,4
8,2,view,1438970905669,342816,None,5
9,2,view,1438971444375,342816,None,6


In [5]:
mysql("""
    SELECT *
    FROM (
        SELECT
            visitorid
            ,event
            ,timestamp
            ,itemid
            ,transactionid
            ,ROW_NUMBER() OVER(PARTITION BY visitorid, event ORDER BY timestamp) seq
        FROM event_df
        GROUP BY 1,2,3,4,5
    ) a
    WHERE seq = 1
    LIMIT 10
    ;
""")

,visitorid,event,timestamp,itemid,transactionid,seq
0,0,view,1442004589439,285930,None,1
1,1,view,1439487966444,72028,None,1
2,2,view,1438969904567,325215,None,1
3,3,view,1438413035296,385090,None,1
4,4,view,1442352267167,177677,None,1
5,5,view,1437097556439,61396,None,1
6,6,addtocart,1440914628202,65273,None,1
7,6,view,1440916778318,253615,None,1
8,7,view,1431581976753,139394,None,1
9,8,view,1433030513812,434230,None,1


In [ ]:
mysql("""select UNIX_TIMESTAMP(timestamp) from event_df limit 10""")

In [ ]:
mysql("""
    SELECT 
        views
        ,cart
        ,COUNT(DISTINCT visitorid) visits
    FROM (
        SELECT
            visitorid
            ,DATE()
            ,MAX(CASE WHEN event = 'view' THEN 1 ELSE 0 END) views
            ,MAX(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) cart
            ,MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) txn
        FROM event_df
        GROUP BY 1
    ) a
    WHERE
        txn = 1
    GROUP BY 1,2
    ;
""")